# Description

## Parameters setting
`data_directory`: path of directory where the data is stored.

In [ ]:
data_directory = 'data_for_paper/'
number_of_individuals_for_train = 32

device = "cuda"
epoch=150000
batch_size=64
model_name = 'regL'
model_dir='model'
lrFE=0.0002
lrSV=0.0002
lrREG=0.0002
lossREG=0.002
beta1=0.5
beta2=0.999

load_dir='model'

: 

## Set-up data

In [ ]:
from lib.data_manager import DataManager
DM = DataManager(path=data_directory)
DM.loads_data()
DM.normalize_data()

## Train model

In [ ]:
from lib.regressor_basic import Regressor
import numpy as np
rndtr = np.random.randint(0, DM.data_input_train.shape[0], number_of_individuals_for_train)

Reg=Regressor(epoch=epoch, batch_size=batch_size,
                model_dir="{}/{}".format(model_dir, model_name),
                lrFE=lrFE, lrSV=lrSV, lrREG=lrREG, lossREG=lossREG,
                beta1=beta1, beta2=beta2)
Reg.train(tr_in = DM.data_input_train[rndtr],
        tr_lb = DM.data_label_train[rndtr],
        ts_in = DM.data_input_test,
        ts_sv = DM.data_severity_test)

## Test model

In [ ]:
from lib.regressor_only_for_test import Regressor
from lib.accuracy_measure import AccuracyMeasure
from lib.bland_altman import BlandAltman
from lib.performance import Performance

Reg = Regressor(device=device)
x_test, y_test, r_test, R_loss_test = Reg.test(
    ts_in = DM.data_input_test, 
    ts_sv = DM.data_severity_test, 
    model_dir = "{}/{}".format(model_dir, model_name),
    epoch = epoch
)

AccuracyMeasure().measures(
    x_test = x_test,
    y_test = y_test, 
    r_test = r_test, 
    x_ref = tr_sv[rndtr], 
    R_loss_test = R_loss_test,
    save_path = 'plot_accuracy/' + model_name + '.png'
)

#%% BA Plot
randint = np.random.randint(0, y_test.shape[0], 16)
BlandAltman().plots(
    y_real = y_test, 
    y_pred = r_test, 
    randint = randint, 
    save_path = 'plot_BA/' + model_name + '.png'
)

#%% Performnace
Performance().measures(
    y_real = y_test,
    y_pred = r_test, 
    roc_path = 'plot_roc/' + model_name + '.png', 
    performance_path = 'performance/' + model_name
)

from lib.regressor_only_for_test import Regressor